<a href="https://colab.research.google.com/github/aruaru0/colab_notebook/blob/main/chaii_qa_5_fold_xlmroberta_torch_fit_multi_sample_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add Multi sample dropout

# for colab

In [ ]:
!nvidia-smi

Fri Nov  5 12:24:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install --upgrade --force-reinstall --no-deps  kaggle > /dev/null
! mkdir /root/.kaggle
! cp "/content/drive/My Drive/Kaggle/kaggle.json" /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c chaii-hindi-and-tamil-question-answering
!kaggle datasets download -d rhtsingh/mlqa-hindi-processed
!kaggle datasets download -d nbroad/xlm-roberta-squad2
!kaggle datasets download -d msafi04/squad-qa-tamil-dataset
!kaggle datasets download -d kishalmandal/cleaned-data-for-chaii
!kaggle datasets download -d harshwalia/chaiiextended-and-cleaned-datasetready-to-train
!kaggle datasets download -d tkm2261/google-translated-squad20-to-hindi-and-tamil

 73% 5.00M/6.81M [00:00<00:00, 28.9MB/s]
100% 6.81M/6.81M [00:00<00:00, 38.1MB/s]
  0% 0.00/2.49M [00:00<?, ?B/s]
100% 2.49M/2.49M [00:00<00:00, 142MB/s]
 99% 2.51G/2.53G [00:19<00:00, 143MB/s]
100% 2.53G/2.53G [00:19<00:00, 140MB/s]
  0% 0.00/564k [00:00<?, ?B/s]
100% 564k/564k [00:00<00:00, 117MB/s]
  0% 0.00/6.71M [00:00<?, ?B/s]
100% 6.71M/6.71M [00:00<00:00, 86.9MB/s]
  0% 0.00/9.29M [00:00<?, ?B/s]
100% 9.29M/9.29M [00:00<00:00, 123MB/s]
 31% 9.00M/29.0M [00:00<00:00, 75.0MB/s]
100% 29.0M/29.0M [00:00<00:00, 150MB/s] 


In [ ]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!mkdir -p input output

In [ ]:
!mkdir input/chaii-hindi-and-tamil-question-answering
!unzip /content/chaii-hindi-and-tamil-question-answering.zip -d input/chaii-hindi-and-tamil-question-answering
!mkdir input/mlqa-hindi-processed
!unzip /content/mlqa-hindi-processed.zip -d input/mlqa-hindi-processed
!mkdir input/xlm-roberta-squad2
!unzip /content/xlm-roberta-squad2.zip -d input/xlm-roberta-squad2/
!mkdir input/squad-qa-tamil-dataset
!unzip /content/squad-qa-tamil-dataset.zip -d input/squad-qa-tamil-dataset
!mkdir input/cleaned-data-for-chaii
!unzip /content/cleaned-data-for-chaii.zip -d input/cleaned-data-for-chaii
!mkdir input/chaiiextended-and-cleaned-datasetready-to-train
!unzip /content/chaiiextended-and-cleaned-datasetready-to-train.zip -d input/chaiiextended-and-cleaned-datasetready-to-train
!mkdir input/google-translated-squad20-to-hindi-and-tamil
!unzip /content/google-translated-squad20-to-hindi-and-tamil.zip -d input/google-translated-squad20-to-hindi-and-tamil

Archive:  /content/chaii-hindi-and-tamil-question-answering.zip
  inflating: input/chaii-hindi-and-tamil-question-answering/sample_submission.csv  
  inflating: input/chaii-hindi-and-tamil-question-answering/test.csv  
  inflating: input/chaii-hindi-and-tamil-question-answering/train.csv  
Archive:  /content/mlqa-hindi-processed.zip
  inflating: input/mlqa-hindi-processed/mlqa_hindi.csv  
  inflating: input/mlqa-hindi-processed/xquad.csv  
Archive:  /content/xlm-roberta-squad2.zip
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2/config.json  
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2/pytorch_model.bin  
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2/sentencepiece.bpe.model  
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2/special_tokens_map.json  
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2/tokenizer.json  
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-bas

In [ ]:
# !mkdir input/chaii
# !kaggle datasets download -d aruaru0/chaii-no006
#!unzip /content/chaii-no006.zip -d input/chaii
#!kaggle datasets download -d nguyenduongthanh/chaii-xlmr-5-fold
#!unzip /content/chaii-xlmr-5-fold.zip -d input/chaii

# restart here..

In [ ]:
%cd output

/content/output


In [ ]:
!pip install transformers

<h2>chaii QA - 5 Fold XLMRoberta Finetuning in Torch w/o Trainer API</h2>
    
<h3><span "style: color=#444">Introduction</span></h3>

The kernel implements 5-Fold XLMRoberta QA Model without using the Trainer API from HuggingFace.

This is a three part kernel,

- [External Data - MLQA, XQUAD Preprocessing](https://www.kaggle.com/rhtsingh/external-data-mlqa-xquad-preprocessing) which preprocesses the Hindi Corpus of MLQA and XQUAD. I have used these data for training.

- [chaii QA - 5 Fold XLMRoberta Torch | FIT](https://www.kaggle.com/rhtsingh/chaii-qa-5-fold-xlmroberta-torch-fit/edit) This kernel is the current kernel and used for Finetuning (FIT) on competition + external data.

- [chaii QA - 5 Fold XLMRoberta Torch | Infer](https://www.kaggle.com/rhtsingh/chaii-qa-5-fold-xlmroberta-torch-infer) The Inference kernel where we ensemble our 5 Fold XLMRoberta Models and do the submission.

<h3><span "style: color=#444">Techniques</span></h3>

The kernel has implementation for below techniques, click on the links to learn more -

 - [External Data Preprocessing + Training](https://www.kaggle.com/rhtsingh/external-data-mlqa-xquad-preprocessing)
 
 - [Mixed Precision Training using APEX](https://www.kaggle.com/rhtsingh/swa-apex-amp-interpreting-transformers-in-torch)
 
 - [Gradient Accumulation](https://www.kaggle.com/rhtsingh/speeding-up-transformer-w-optimization-strategies)
 
 - [Gradient Clipping](https://www.kaggle.com/rhtsingh/swa-apex-amp-interpreting-transformers-in-torch)
 
 - [Grouped Layerwise Learning Rate Decay](https://www.kaggle.com/rhtsingh/guide-to-huggingface-schedulers-differential-lrs)
 
 - [Utilizing Intermediate Transformer Representations](https://www.kaggle.com/rhtsingh/utilizing-transformer-representations-efficiently)
 
 - [Layer Initialization](https://www.kaggle.com/rhtsingh/on-stability-of-few-sample-transformer-fine-tuning)
 
 - [5-Fold Training](https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-fit)
 
 - etc.
 
<h3><span "style: color=#444">References</span></h3>
I would like to acknowledge below kagglers work and resources without whom this kernel wouldn't have been possible,  

- [roberta inference 5 folds by Abhishek](https://www.kaggle.com/abhishek/roberta-inference-5-folds)

- [ChAII - EDA & Baseline by Darek](https://www.kaggle.com/thedrcat/chaii-eda-baseline) due to whom i found the great notebook below from HuggingFace,

- [How to fine-tune a model on question answering](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb)

- [HuggingFace QA Examples](https://github.com/huggingface/transformers/tree/master/examples/pytorch/question-answering)

- [TensorFlow 2.0 Question Answering - Collections of gold solutions](https://www.kaggle.com/c/tensorflow2-question-answering/discussion/127409) these solutions are gold and highly recommend everyone to give a detailed read.

<h3><span style="color=#444">Note</span></h3>

The below points are worth noting,

 - I haven't used FP16 because due to some reason this fails and model never starts training.
 - These are the original hyperparamters and setting that I have used for training my models.
 - I tried few pooling layers but none of them performed better than simple one.
 - Gradient clipping reduces model performance.
 - <span style="color:#2E86C1">Training 5 Folds at once will give OOM issue on Kaggle and Colab. I have trained one fold at a time i.e. After 1st fold is completed I save the model as a dataset then train second fold. Repeat this process until all 5 folds are completed. Training each fold takes around 50 minuts on Kaggle.</span>
 - <span style="color:#2E86C1">I have modified the preprocessing code a lot from original used in HuggingFace notebook since I am not using HuggingFace Datasets API as well. So I had to handle the sequence-ids specially since they contain None values which torch doesn't support.</span>

### Install APEX

In [ ]:
# %%writefile setup.sh
# export CUDA_HOME=/usr/local/cuda-10.1
# git clone https://github.com/NVIDIA/apex
# cd apex
# pip install -v --disable-pip-version-check --no-cache-dir ./

In [ ]:
# %%capture
# !sh setup.sh

### Import Dependencies

In [ ]:
import os
import gc
gc.enable()
import math
import json
import time
import random
import multiprocessing
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import torch.optim as optim
from torch.utils.data import (
    Dataset, DataLoader,
    SequentialSampler, RandomSampler
)
from torch.utils.data.distributed import DistributedSampler

try:
    from apex import amp
    APEX_INSTALLED = True
except ImportError:
    APEX_INSTALLED = False

import transformers
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule_with_warmup,
    logging,
    MODEL_FOR_QUESTION_ANSWERING_MAPPING,
)
logging.set_verbosity_warning()
logging.set_verbosity_error()

def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def optimal_num_of_loader_workers():
    num_cpus = multiprocessing.cpu_count()
    num_gpus = torch.cuda.device_count()
    optimal_value = min(num_cpus, num_gpus*4) if num_gpus else num_cpus - 1
    return optimal_value

print(f"Apex AMP Installed :: {APEX_INSTALLED}")
MODEL_CONFIG_CLASSES = list(MODEL_FOR_QUESTION_ANSWERING_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

Apex AMP Installed :: False


### Training Configuration

tamil & hindi xsquad

In [ ]:
class Config:
    # model
    model_type = 'xlm_roberta'
    #model_name_or_path = "deepset/xlm-roberta-large-squad2"
    model_name_or_path = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"
    #config_name = "deepset/xlm-roberta-large-squad2"
    config_name = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"
    fp16 = True if APEX_INSTALLED else False
    fp16_opt_level = "O1"
    
    gradient_accumulation_steps =  8#@param {type:"integer"}

    # tokenizer
    tokenizer_name = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"
#    max_seq_length = 384
#    doc_stride = 128
    max_seq_length =  400#@param {type:"integer"}
    doc_stride =  135#@param {type:"integer"}

    # train
    epochs =  1#@param {type:"integer"}
    train_batch_size = 4
    eval_batch_size = 8

    # optimizer
    optimizer_type = 'AdamW'
    learning_rate =  2e-5#@param {type:"number"}
    weight_decay =   1e-3#@param {type:"number"}
    epsilon = 1e-10 #@param {type:"number"}
    max_grad_norm = 1.0 #@param {type:"number"}

    # scheduler
    decay_name = 'linear-warmup'
    warmup_ratio = 0.1
    scheduler_step = 2#@param {type:"integer"}

    # logging
    ####logging_steps = 10
    logging_steps = 1000

    # evaluate
    output_dir = 'output'
    seed =  42#@param {type:"integer"}

print("acc_step", Config.gradient_accumulation_steps,
      "lr", Config.learning_rate, "wd", Config.weight_decay,
      "{}-{}".format(Config.max_seq_length, Config.doc_stride),
      "seed", Config.seed)

acc_step 8 lr 2e-05 wd 0.001 400-135 seed 42


### Data Factory

In [ ]:
#external_tamil_xquad = pd.read_csv('/content/input/squad-qa-tamil-dataset/squad_tamilQA.csv')
#external_xquad = pd.read_csv('../input/mlqa-hindi-processed/xquad.csv')


In [ ]:
#external_tamil_xquad['language'] = "tamil"
#external_tamil_xquad.head()

In [ ]:
import ast
external_hindi_xquad2 = pd.read_csv('/content/input/google-translated-squad20-to-hindi-and-tamil/squad_hi.csv')

text, start = [], []
for i in range(len(external_hindi_xquad2)):
  ans = external_hindi_xquad2['answers'][i]
  dic = ast.literal_eval(ans)[0]
  text.append(dic['text'])
  start.append(dic['answer_start'])

external_hindi_xquad2['answer_text'] = text
external_hindi_xquad2['answer_start'] = start
external_hindi_xquad2['language'] = "hindi"

external_hindi_xquad2 = external_hindi_xquad2.drop(['id', 'answers','c_id','is_in'], axis=1)
external_hindi_xquad2 = external_hindi_xquad2.sample(frac=1, random_state=42)[:20000]

external_hindi_xquad2.head()

,question,context,answer_text,answer_start,language
46160,चर्चों की विश्व परिषद ने किस प्रकार के धर्मशास...,"1965 में, पॉल VI ने विश्व चर्च परिषद के साथ एक...",कैथोलिक,868,hindi
55825,थाईलैंड में संविधान के पालन को क्या नुकसान पहु...,"थाईलैंड में, एक ऐसा राज्य जिसके पास १९३२ में प...",प्राचीन पूर्वाग्रह और राजनीतिक पूर्वाग्रह,220,hindi
13361,एमसीए ने यूनिवर्सल का अधिग्रहण किस वर्ष किया था?,"एमसीए, इंक. द्वारा यूनिवर्सल पिक्चर्स का लंबे ...",1962,73,hindi
15969,अन्ना लेल्केस ने कौन सा वाद्य यंत्र बजाया?,"1997 में, विएना फिलहारमोनिक को राष्ट्रीय महिला...",वीणा,385,hindi
10658,हैलिफ़ैक्स विस्फोट किस दिन हुआ था?,"अपने जीवन के अंत तक, बेल और उनका परिवार दो घरो...","6 दिसंबर, 1917",394,hindi


In [ ]:
external_tamil_xquad2 = pd.read_csv('/content/input/google-translated-squad20-to-hindi-and-tamil/squad_ta.csv')

text, start = [], []
for i in range(len(external_tamil_xquad2)):
  ans = external_tamil_xquad2['answers'][i]
  dic = ast.literal_eval(ans)[0]
  text.append(dic['text'])
  start.append(dic['answer_start'])

external_tamil_xquad2['answer_text'] = text
external_tamil_xquad2['answer_start'] = start
external_tamil_xquad2['language'] = "tamil"

external_tamil_xquad2 = external_tamil_xquad2.drop(['id', 'answers','c_id','is_in'], axis=1)
external_tamil_xquad2 = external_tamil_xquad2.sample(frac=1, random_state=123)[:10000]
external_tamil_xquad2.head()

,question,context,answer_text,answer_start,language
3148,மன்ஹாட்டன் ஜெனரல் பீட்டர் மினூட்டால் வாங்கப்பட...,நியூ நெதர்லாந்தில் ஒரு நிரந்தர ஐரோப்பிய இருப்ப...,கண்ணாடி மணிகள்,-1,tamil
25303,2014 இல் சராசரி இம்பீரியல் பட்டதாரி எந்த புள்ள...,"மேலும், வேலை வாய்ப்புகளைப் பொறுத்தவரை, 2014 ஆம...",சராசரி தொடக்க சம்பளம்,89,tamil
54729,CETDZ இன் முதலீட்டாளர்கள் மற்றும் டெவலப்பர்களா...,செங்டு பொருளாதார மற்றும் தொழில்நுட்ப மேம்பாட்ட...,20,123,tamil
42221,காமன்வெல்த் மன்னர் யார்?,"20 ஆம் நூற்றாண்டின் தொடக்கத்தில், ஜெர்மனியும் ...",ராணி இரண்டாம் எலிசபெத்,-1,tamil
31696,கின்ஸி வேலை சரியில்லை என்று மக்கள் ஏன் சொல்கிற...,"ஆல்ஃபிரட் சி. கின்சி மற்றும் பலர், மனித ஆண் (1...","அவரது மாதிரி மக்கள்தொகையின் சீரற்ற தன்மை,",-1,tamil


In [ ]:
train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
external_mlqa = pd.read_csv('../input/mlqa-hindi-processed/mlqa_hindi.csv')
external_train = pd.concat([external_mlqa, external_hindi_xquad2, external_tamil_xquad2]).reset_index(drop=True)

def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021) # 2021)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data['language'])):
        data.loc[v_, 'kfold'] = f
    return data

train = create_folds(train, num_splits=5)
external_train["kfold"] = -1
external_train['id'] = list(np.arange(1, len(external_train)+1))

train = pd.concat([train, external_train]).reset_index(drop=True)

def convert_answers(row):
    return {'answer_start': [row[0]], 'text': [row[1]]}

train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [ ]:
print(len(train),len(train['id'].unique()))

36539 36539


In [ ]:
# train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
# # train = pd.read_csv('../input/cleaned-data-for-chaii/cleaned_train.csv')
# # train = train[train['language']=='tamil'].reset_index(drop=True)
# #train = pd.read_csv('/content/input/chaiiextended-and-cleaned-datasetready-to-train/ChaiiExtended.csv')
# #train['language'] = "xxxx"

# test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
# external_mlqa = pd.read_csv('../input/mlqa-hindi-processed/mlqa_hindi.csv')
# external_hindi_xquad = pd.read_csv('../input/mlqa-hindi-processed/xquad.csv')
# external_tamil_xquad = pd.read_csv('/content/input/squad-qa-tamil-dataset/squad_tamilQA.csv')
# external_tamil_xquad['language'] = "tamil"
# external_xquad = pd.concat([external_hindi_xquad, external_tamil_xquad])
# external_train = external_hindi_xquad #pd.concat([external_mlqa, external_xquad]).reset_index(drop=True)

# def create_folds(data, num_splits):
#     data["kfold"] = -1
#     kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=6654) # 2021)
#     for f, (t_, v_) in enumerate(kf.split(X=data, y=data['language'])):
#         data.loc[v_, 'kfold'] = f
#     return data

# #####train = create_folds(train, num_splits=5)
# train = create_folds(train, num_splits=2)
# external_train["kfold"] = -1
# external_train['id'] = list(np.arange(1, len(external_train)+1))
# #external_train = create_folds(external_train, num_splits=3) # foldに分割（2021.10.09変更）
# #external_train["kfold"] = external_train["kfold"] + 10

# train = pd.concat([train, external_train]).reset_index(drop=True)

# def convert_answers(row):
#     return {'answer_start': [row[0]], 'text': [row[1]]}

# train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [ ]:
train.head()

,id,context,question,answer_text,answer_start,language,kfold,answers
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,3,"{'answer_start': [53], 'text': ['206']}"
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0,"{'answer_start': [2358], 'text': ['காசுமீரில்']}"
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,4,"{'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட..."
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,4,"{'answer_start': [68], 'text': ['தாலாட்டு']}"
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,4,"{'answer_start': [585], 'text': ['சூரியனும்']}"


In [ ]:
if 0:
    train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
    test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
    external_mlqa = pd.read_csv('../input/mlqa-hindi-processed/mlqa_hindi.csv')
    external_hindi_xquad = pd.read_csv('../input/mlqa-hindi-processed/xquad.csv')
    external_tamil_xquad = pd.read_csv('/content/input/squad-qa-tamil-dataset/squad_tamilQA.csv')
    external_tamil_xquad['language'] = "tamil"
    external_xquad = pd.concat([external_hindi_xquad, external_tamil_xquad])
    external_train = external_hindi_xquad # pd.concat([external_mlqa, external_xquad])

    def create_folds(data, num_splits):
        data["kfold"] = -1
        kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021)
        for f, (t_, v_) in enumerate(kf.split(X=data, y=data['language'])):
            data.loc[v_, 'kfold'] = f
        return data

    train = create_folds(train, num_splits=5)
    external_train["kfold"] = -1
    external_train['id'] = list(np.arange(1, len(external_train)+1))
    train = pd.concat([train, external_train]).reset_index(drop=True)

    def convert_answers(row):
        return {'answer_start': [row[0]], 'text': [row[1]]}

    train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

### Covert Examples to Features (Preprocess)

In [ ]:
def prepare_train_features(args, example, tokenizer):
    example["question"] = example["question"].lstrip()
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=args.max_seq_length,
        stride=args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_example.pop("offset_mapping")

    features = []
    for i, offsets in enumerate(offset_mapping):
        feature = {}

        input_ids = tokenized_example["input_ids"][i]
        attention_mask = tokenized_example["attention_mask"][i]

        feature['input_ids'] = input_ids
        feature['attention_mask'] = attention_mask
        feature['offset_mapping'] = offsets

        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_example.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = example["answers"]

        if len(answers["answer_start"]) == 0:
            feature["start_position"] = cls_index
            feature["end_position"] = cls_index
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                feature["start_position"] = cls_index
                feature["end_position"] = cls_index
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                feature["start_position"] = token_start_index - 1
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                feature["end_position"] = token_end_index + 1

        features.append(feature)
    return features

### Dataset Retriever

In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, features, mode='train'):
        super(DatasetRetriever, self).__init__()
        self.features = features
        self.mode = mode
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, item):   
        feature = self.features[item]
        if self.mode == 'train':
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

### Model

In [ ]:
class Model(nn.Module):
    def __init__(self, modelname_or_path, config):
        super(Model, self).__init__()
        self.n_msd = 5
        self.config = config
        self.xlm_roberta = AutoModel.from_pretrained(modelname_or_path, config=config)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        self.dropout = nn.Dropout(0.1)
        # self.dropouts = nn.ModuleList([nn.Dropout(0.1) for _ in range(self.n_msd)])
        self._init_weights(self.qa_outputs)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
        # token_type_ids=None
    ):
        outputs = self.xlm_roberta(
            input_ids,
            attention_mask=attention_mask,
        )

        sequence_output = outputs[0]
        pooled_output = outputs[1]
        
        sequence_output = self.dropout(sequence_output)
        qa_logits = self.qa_outputs(sequence_output)
        # qa_logits = sum([self.qa_outputs(dropout(sequence_output)) for dropout in self.dropouts])/self.n_msd
        #print(qa_logits.size())
        start_logits, end_logits = qa_logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
        return start_logits, end_logits

### Loss

In [ ]:
def compute_kl_loss(p, q, pad_mask=None):
    start_preds1, end_preds1 = p
    start_preds2, end_preds2 = q

    p_loss1 = F.kl_div(F.log_softmax(start_preds1, dim=-1), F.softmax(start_preds2, dim=-1), reduction='none')
    q_loss1 = F.kl_div(F.log_softmax(start_preds2, dim=-1), F.softmax(start_preds1, dim=-1), reduction='none')
    p_loss2 = F.kl_div(F.log_softmax(end_preds1, dim=-1), F.softmax(end_preds2, dim=-1), reduction='none')
    q_loss2 = F.kl_div(F.log_softmax(end_preds2, dim=-1), F.softmax(end_preds1, dim=-1), reduction='none')

    # You can choose whether to use function "sum" and "mean" depending on your task
    p_loss = p_loss1.sum() + p_loss2.sum() 
    q_loss = q_loss1.sum() + q_loss2.sum()

    loss = (p_loss + q_loss) / 2
    return loss

In [ ]:
def linear_combination(x, y, epsilon):
    return (1 - epsilon) * x + epsilon * y

def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction == 'mean' else loss.sum() if reduction == 'sum' else loss

In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon=0.1, ignore_index = -1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
        self.ignore_index = ignore_index

    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction, ignore_index=self.ignore_index)
        return linear_combination(nll, loss/n, self.epsilon)

In [ ]:
def loss_fn(preds, labels):
    start_preds, end_preds = preds
    start_labels, end_labels = labels
    
    start_loss = nn.CrossEntropyLoss(ignore_index=-1)(start_preds, start_labels)
    end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_preds, end_labels)
    #start_loss = LabelSmoothingCrossEntropy(ignore_index=-1)(start_preds, start_labels)
    #end_loss = LabelSmoothingCrossEntropy(ignore_index=-1)(end_preds, end_labels)
    total_loss = (start_loss + end_loss) / 2
    return total_loss

In [ ]:
def loss_fn2(pred1, pred2, label):
    # RMSE loss
    loss1 = loss_fn(pred1, label)
    loss2 = loss_fn(pred2, label)

    # R-Drop
    kl_loss = compute_kl_loss(pred1, pred2)

    RDROP_ALPHA = 0.1
    total_loss = (loss1 + loss2) / 2 + RDROP_ALPHA * kl_loss
    return total_loss


### Grouped Layerwise Learning Rate Decay

In [ ]:
# def make_model(args):
#     config = AutoConfig.from_pretrained(args.config_name)
#     tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
#     model = Model(args.model_name_or_path, config=config)
#     return config, tokenizer, model

# cfg, tk, m = make_model(Config)

In [ ]:
# x  = get_optimizer_grouped_parameters(Config(), m)

In [ ]:
def get_optimizer_grouped_parameters(args, model):
    model_type = 'xlm_roberta'
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters()
                       if 'qa_outputs' in n],
            "weight_decay": 0.0,
            "lr": 1e-4 ####### args.learning_rate*5
        },
    ]
    # print(optimizer_grouped_parameters)
    num_layers = model.config.num_hidden_layers
    layers = [getattr(model, model_type).embeddings] + list(getattr(model, model_type).encoder.layer)
    layers.reverse()
    lr = args.learning_rate
    LR_DECAY = 0.95
    for layer in layers:
        lr *= LR_DECAY
        optimizer_grouped_parameters += [
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": args.weight_decay,
                "lr": lr,
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": lr,
            },
        ]
    return optimizer_grouped_parameters


# def get_optimizer_grouped_parameters(args, model):
#     no_decay = ["bias", "LayerNorm.weight"]
#     group1=['layer.0.','layer.1.','layer.2.','layer.3.']
#     group2=['layer.4.','layer.5.','layer.6.','layer.7.']    
#     group3=['layer.8.','layer.9.','layer.10.','layer.11.']
#     group_all=['layer.0.','layer.1.','layer.2.','layer.3.','layer.4.','layer.5.','layer.6.','layer.7.','layer.8.','layer.9.','layer.10.','layer.11.']
#     optimizer_grouped_parameters = [
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': args.weight_decay},
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': args.weight_decay, 'lr': args.learning_rate/8},
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': args.weight_decay, 'lr': args.learning_rate/4},
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': args.weight_decay, 'lr': args.learning_rate},
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': 0.0},
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': 0.0, 'lr': args.learning_rate/8},
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': 0.0, 'lr': args.learning_rate/4},
#         {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': 0.0, 'lr': args.learning_rate},
#         {'params': [p for n, p in model.named_parameters() if args.model_type not in n], 'lr':args.learning_rate*20, "weight_decay": 0.0},
#     ]
#     return optimizer_grouped_parameters

In [ ]:
if 0:
  def get_optimizer_grouped_parameters(args, model):
    no_decay = ["bias", "LayerNorm.weight"]
    group1=['layer.0.','layer.1.','layer.2.','layer.3.']
    group2=['layer.4.','layer.5.','layer.6.','layer.7.']    
    group3=['layer.8.','layer.9.','layer.10.','layer.11.']
    group_all=['layer.0.','layer.1.','layer.2.','layer.3.','layer.4.','layer.5.','layer.6.','layer.7.','layer.8.','layer.9.','layer.10.','layer.11.']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': args.weight_decay, 'lr': args.learning_rate/2.6},
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': args.weight_decay, 'lr': args.learning_rate},
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': args.weight_decay, 'lr': args.learning_rate*2.6},
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': 0.0},
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': 0.0, 'lr': args.learning_rate/2.6},
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': 0.0, 'lr': args.learning_rate},
        {'params': [p for n, p in model.xlm_roberta.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': 0.0, 'lr': args.learning_rate*2.6},
        {'params': [p for n, p in model.named_parameters() if args.model_type not in n], 'lr':args.learning_rate*20, "weight_decay": 0.0},
    ]
    return optimizer_grouped_parameters

### Metric Logger

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.max = 0
        self.min = 1e5

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        if val > self.max:
            self.max = val
        if val < self.min:
            self.min = val

### Utilities

In [ ]:
def make_model(args):
    config = AutoConfig.from_pretrained(args.config_name)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
    model = Model(args.model_name_or_path, config=config)
    return config, tokenizer, model

def make_optimizer(args, model):
    #################
    optimizer_grouped_parameters = get_optimizer_grouped_parameters(args, model)
    # no_decay = ["bias", "LayerNorm.weight"]
    # optimizer_grouped_parameters = [
    #     {
    #         "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
    #         "weight_decay": args.weight_decay,
    #     },
    #     {
    #         "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
    #         "weight_decay": 0.0,
    #     },
    # ]
    if args.optimizer_type == "AdamW":
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            eps=args.epsilon,
            correct_bias=True
        )
        return optimizer

def make_scheduler(
    args, optimizer, 
    num_warmup_steps, 
    num_training_steps
):
    if args.decay_name == "cosine-warmup":
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps
        )
    elif args.decay_name == "liner-warmup":
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps
        )
    else:
        scheduler = get_constant_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
        )
    return scheduler    

def make_loader(
    args, data, 
    tokenizer, fold
):
    train_set, valid_set = data[data['kfold']!=fold], data[data['kfold']==fold]
    
    train_features, valid_features = [[] for _ in range(2)]
    for i, row in train_set.iterrows():
        train_features += prepare_train_features(args, row, tokenizer)
    for i, row in valid_set.iterrows():
        valid_features += prepare_train_features(args, row, tokenizer)

    train_dataset = DatasetRetriever(train_features)
    valid_dataset = DatasetRetriever(valid_features)
    print(f"Num examples Train= {len(train_dataset)}, Num examples Valid={len(valid_dataset)}")
    
    train_sampler = RandomSampler(train_dataset)
    valid_sampler = SequentialSampler(valid_dataset)

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=args.train_batch_size,
        sampler=train_sampler,
        num_workers=optimal_num_of_loader_workers(),
        pin_memory=True,
        drop_last=False 
    )

    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=args.eval_batch_size, 
        sampler=valid_sampler,
        num_workers=optimal_num_of_loader_workers(),
        pin_memory=True, 
        drop_last=False
    )

    return train_dataloader, valid_dataloader

### Trainer

In [ ]:
class Trainer:
    def __init__(
        self, model, tokenizer, 
        optimizer, scheduler
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = optimizer
        self.scheduler = scheduler

    def train(
        self, args, 
        train_dataloader, 
        epoch, result_dict
    ):
        count = 0
        losses = AverageMeter()
        
        self.model.zero_grad()
        self.model.train()
        
        fix_all_seeds(args.seed)
        
        for batch_idx, batch_data in enumerate(train_dataloader):
            input_ids, attention_mask, targets_start, targets_end = \
                batch_data['input_ids'], batch_data['attention_mask'], \
                    batch_data['start_position'], batch_data['end_position']
            
            input_ids, attention_mask, targets_start, targets_end = \
                input_ids.cuda(), attention_mask.cuda(), targets_start.cuda(), targets_end.cuda()

            outputs_start, outputs_end = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )
            
            ########
            # outputs_start2, outputs_end2 = self.model(
            #     input_ids=input_ids,
            #     attention_mask=attention_mask,
            # )

            #print(outputs_start.size())
            loss = loss_fn((outputs_start, outputs_end), (targets_start, targets_end))
            # loss = loss_fn2((outputs_start, outputs_end), (outputs_start2, outputs_end2), (targets_start, targets_end))
            loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            count += input_ids.size(0)
            losses.update(loss.item(), input_ids.size(0))

            # if args.fp16:
            #     torch.nn.utils.clip_grad_norm_(amp.master_params(self.optimizer), args.max_grad_norm)
            # else:
            #     torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.max_grad_norm)

            if batch_idx % args.gradient_accumulation_steps == 0 or batch_idx == len(train_dataloader) - 1:
                self.optimizer.step()
                self.scheduler.step()
                self.optimizer.zero_grad()

            if (batch_idx % args.logging_steps == 0) or (batch_idx+1)==len(train_dataloader):
                _s = str(len(str(len(train_dataloader.sampler))))
                ret = [
                    ('Epoch: {:0>2} [{: >' + _s + '}/{} ({: >3.0f}%)]').format(epoch, count, len(train_dataloader.sampler), 100 * count / len(train_dataloader.sampler)),
                    'Train Loss: {: >4.5f}'.format(losses.avg),
                ]
                print(', '.join(ret))

        result_dict['train_loss'].append(losses.avg)
        return result_dict

### Evaluator

In [ ]:
class Evaluator:
    def __init__(self, model):
        self.model = model
    
    def save(self, result, output_dir):
        with open(f'{output_dir}/result_dict.json', 'w') as f:
            f.write(json.dumps(result, sort_keys=True, indent=4, ensure_ascii=False))

    def evaluate(self, valid_dataloader, epoch, result_dict):
        losses = AverageMeter()
        for batch_idx, batch_data in enumerate(valid_dataloader):
            self.model = self.model.eval()
            input_ids, attention_mask, targets_start, targets_end = \
                batch_data['input_ids'], batch_data['attention_mask'], \
                    batch_data['start_position'], batch_data['end_position']
            
            input_ids, attention_mask, targets_start, targets_end = \
                input_ids.cuda(), attention_mask.cuda(), targets_start.cuda(), targets_end.cuda()
            
            with torch.no_grad():            
                outputs_start, outputs_end = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                )
                
                loss = loss_fn((outputs_start, outputs_end), (targets_start, targets_end))
                losses.update(loss.item(), input_ids.size(0))
                
        print('----Validation Results Summary----')
        print('Epoch: [{}] Valid Loss: {: >4.5f}'.format(epoch, losses.avg))
        result_dict['val_loss'].append(losses.avg)        
        return result_dict

### Initialize Training

In [ ]:
def init_training(args, data, fold):
    fix_all_seeds(args.seed)
    
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    
    # model
    model_config, tokenizer, model = make_model(args)
    if torch.cuda.device_count() >= 1:
        print('Model pushed to {} GPU(s), type {}.'.format(
            torch.cuda.device_count(), 
            torch.cuda.get_device_name(0))
        )
        model = model.cuda() 
    else:
        raise ValueError('CPU training is not supported')
    
    # print("load dict...")
    # model.load_state_dict(
    #     torch.load("/content/input/chaii/output/checkpoint-fold-{}/pytorch_model.bin".format(fold))
    #     #torch.load("/content/drive/MyDrive/datas/chaii/output/output/checkpoint-fold-{}/pytorch_model.bin".format(fold))
    # );

    # data loaders
    train_dataloader, valid_dataloader = make_loader(args, data, tokenizer, fold)

    # optimizer
    optimizer = make_optimizer(args, model)

    # scheduler
    num_training_steps = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps) * args.epochs
    if args.warmup_ratio > 0:
        num_warmup_steps = int(args.warmup_ratio * num_training_steps)
    else:
        num_warmup_steps = 0
    print(f"Total Training Steps: {num_training_steps}, Total Warmup Steps: {num_warmup_steps}")
    scheduler = make_scheduler(args, optimizer, num_warmup_steps, num_training_steps)

    # mixed precision training with NVIDIA Apex
    if args.fp16:
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)
    
    result_dict = {
        'epoch':[], 
        'train_loss': [], 
        'val_loss' : [], 
        'best_val_loss': np.inf
    }

    return (
        model, model_config, tokenizer, optimizer, scheduler, 
        train_dataloader, valid_dataloader, result_dict
    )

### Run

In [ ]:
def run(data, fold):
    args = Config()
    model, model_config, tokenizer, optimizer, scheduler, train_dataloader, \
        valid_dataloader, result_dict = init_training(args, data, fold)
    
    trainer = Trainer(model, tokenizer, optimizer, scheduler)
    evaluator = Evaluator(model)

    train_time_list = []
    valid_time_list = []

    for epoch in range(args.epochs):
        result_dict['epoch'].append(epoch)

        # Train
        torch.cuda.synchronize()
        tic1 = time.time()
        result_dict = trainer.train(
            args, train_dataloader, 
            epoch, result_dict
        )
        torch.cuda.synchronize()
        tic2 = time.time() 
        train_time_list.append(tic2 - tic1)
        
        # Evaluate
        torch.cuda.synchronize()
        tic3 = time.time()
        result_dict = evaluator.evaluate(
            valid_dataloader, epoch, result_dict
        )
        torch.cuda.synchronize()
        tic4 = time.time() 
        valid_time_list.append(tic4 - tic3)
            
        output_dir = os.path.join(args.output_dir, f"checkpoint-fold-{fold}")
        if result_dict['val_loss'][-1] < result_dict['best_val_loss']:
            print("{} Epoch, Best epoch was updated! Valid Loss: {: >4.5f}".format(epoch, result_dict['val_loss'][-1]))
            result_dict["best_val_loss"] = result_dict['val_loss'][-1]        
            
            os.makedirs(output_dir, exist_ok=True)
            torch.save(model.state_dict(), f"{output_dir}/pytorch_model.bin")
            model_config.save_pretrained(output_dir)
            tokenizer.save_pretrained(output_dir)
            print(f"Saving model checkpoint to {output_dir}.")
            
        print()

    evaluator.save(result_dict, output_dir)
    
    print(f"Total Training Time: {np.sum(train_time_list)}secs, Average Training Time per Epoch: {np.mean(train_time_list)}secs.")
    print(f"Total Validation Time: {np.sum(valid_time_list)}secs, Average Validation Time per Epoch: {np.mean(valid_time_list)}secs.")
    
    best = result_dict["best_val_loss"]

    torch.cuda.empty_cache()
    del trainer, evaluator
    del model, model_config, tokenizer
    del optimizer, scheduler
    del train_dataloader, valid_dataloader, result_dict
    gc.collect()

    return best

In [ ]:
best = []
for fold in range(5):
    # if fold >= 2 :
    #   continue
    print();print()
    print('-'*50)
    print(f'FOLD: {fold}')
    print('-'*50)
    ret = run(train, fold)
    !cp -r ./output /content/drive/MyDrive/datas/chaii/output
    best.append(ret)

print(best)



--------------------------------------------------
FOLD: 0
--------------------------------------------------
Model pushed to 1 GPU(s), type Tesla P100-PCIE-16GB.
Num examples Train= 50006, Num examples Valid=2972
Total Training Steps: 1563, Total Warmup Steps: 156
Epoch: 00 [    4/50006 (  0%)], Train Loss: 0.73960
Epoch: 00 [ 4004/50006 (  8%)], Train Loss: 0.28606
Epoch: 00 [ 8004/50006 ( 16%)], Train Loss: 0.21505
Epoch: 00 [12004/50006 ( 24%)], Train Loss: 0.19141
Epoch: 00 [16004/50006 ( 32%)], Train Loss: 0.17662
Epoch: 00 [20004/50006 ( 40%)], Train Loss: 0.16821
Epoch: 00 [24004/50006 ( 48%)], Train Loss: 0.16254
Epoch: 00 [28004/50006 ( 56%)], Train Loss: 0.15789
Epoch: 00 [32004/50006 ( 64%)], Train Loss: 0.15435
Epoch: 00 [36004/50006 ( 72%)], Train Loss: 0.15159
Epoch: 00 [40004/50006 ( 80%)], Train Loss: 0.14901
Epoch: 00 [44004/50006 ( 88%)], Train Loss: 0.14709
Epoch: 00 [48004/50006 ( 96%)], Train Loss: 0.14536
Epoch: 00 [50006/50006 (100%)], Train Loss: 0.14457
----

In [ ]:
np.mean(np.array(best))

0.24418047854444436

## ここをチェック

In [ ]:
# trainデータなし学習 valideデータはtrain.csvではないので注意
#[1.2976857284959564, 1.3881498964859058, 1.3087206599546626, 1.2901440256843426, 1.1679372001281267]
#[0.24349995289113208, 0.2663524366251205, 0.19786230920665912, 0.2553937918101003, 0.18950824988041495]
#[0.24366667471828746, 0.28473904721862214, 0.15692045553454106, 0.24553805843296678, 0.18429825624839946]
#[0.24311262005966766, 0.284736886900765, 0.15840444342909, 0.24527732736873206, 0.18645570367727277]
#[0.23844826735717128, 0.28548798921814356, 0.15855387426083375, 0.24254149177365683, 0.18242730208896807]
#[0.23633872994702976, 0.2876898618870076]

#[0.2399075371348521, 0.21504926022690454, 0.28804823179640204]

#[0.2322873271281629, 0.2690950131782601, 0.20884554429639518, 0.25611597623857196, 0.2622648145021154]


#tamil
#[0.24800505987951157, 0.28482119431719183, 0.15574497073223997, 0.24578075964718757, 0.17901662780208277]



# train & hindi xquad
#[0.24965187649773146, 0.23444136391208764, 0.22017900116515676] 0.23475741385832527
#[0.23096489540942408, 0.28216897352464915, 0.2203173596892458, 0.28736751974058533, 0.208414323353315] 0.2458466143434439
#[0.22519750105769248, 0.25939778196668173, 0.20381177481000717, 0.2465142556059104, 0.17124023813670544] 0.22123231031539947
#[0.2697170414320415, 0.3059851324188871, 0.2551263245063695, 0.3056244693620248, 0.21533218981467775] 0.2703570315068001
#[0.22526602439154889, 0.26299701667625947, 0.20623224054625405, 0.26411216159941286, 0.182503771575793] 0.22822224295785362
#[0.23691583103636119, 0.2686926209007856, 0.2075012739026476, 0.25571625346175986, 0.19657193895548936]
#[0.23888897450804394, 0.26591575717939314, 0.19279731265472355, 0.275491834834653, 0.1854117791257561]
#[0.2519814316728135, 0.26924624716331114, 0.19539388123118323, 0.27420934634778854, 0.184092709692541]
#[0.2530554625445571, 0.27373010504284945, 0.20456079005861058, 0.28659096361988684, 0.20325152608371191]

# ★★ここをベースに検討★★
# hindi xsquid acc_step 2 lr 1e-05 wd 0.001 400-135 seed 42 msd 5
# [0.2302274905697632, 0.27069914932622796, 0.21486875825171511, 0.2573498566401873, 0.18481068504706707] 0.23159118796699213 LB:0.768
#-----------
# hindi xsquid acc_step 2 lr 1e-05 wd 0.001 400-135 seed 42 msd 5 scheduler_step 2 FOLDデータが若干違う　10/12 chaii-1012
#[0.24841012325690787, 0.26584840404003096, 0.2032461542127646, 0.2737542687907607, 0.22716575367217018] LB:0.776
#-----------
# hindi xsquid acc_step 2 lr 1.5e-05 wd 0.001 400-135 seed 42 msd 5 scheduler_step 2 FOLDデータが若干違う　10/13 chaii-1013
# [0.24829580425794023, 0.2651147856025376, 0.21895726531102902, 5.991464610776605, 0.228712301645232] LB:0.778
#      FOLD3 単独でやり直し（学習ミスのため）　1e-5 s-step3に変更 fold3=0.25424
#-----------
# hindi xsquid acc_step 2 lr 1e-05 wd 0.001 400-135 seed 42 msd 5 scheduler_step 3 FOLDデータが若干違う　※10/13が収束しなかったので変更
# [0.23505390209724317, 0.2598879741057279, 0.20111809600251632, 0.2542404424046021, 0.18050300175809506] chaii-1014 LB:0.776

# hindi xsquid acc_step 2 lr 9e-06 wd 0.001 400-135 seed 42 msd 5 scheduler_step 3 FOLDデータが若干違う　10/14
# [0.23501111353810378, 0.2556940006469886, 0.19985474928692182, 0.256803787264871, 0.18670242301902526] chaii-2014-2 LB:0.765

# hindi xsquid acc_step 2 lr 1.5e-05 wd 0.001 400-135 seed 923 msd 5 scheduler_step 2 FOLDデータが若干違う 10/14-2
#[0.22942558399286914, 0.27648799994582035, 0.20045122814375774, 0.2518382845039409, 0.1895024780733636]





In [ ]:
# [0.2639688412511144, 0.23578763837464295, 0.2082770317994482] tamil & hindi xsquad  lr = 1.5e-5 LB:0.758
# [0.2692653296424045, 0.23645380146391437, 0.21420694968205684] hindi xsquad only  lr = 1.5e-5 LB:0.761
# [0.3102661463084304, 0.23890210809797524, 0.22127942127638148] tamil hindi xsquad + hindi mlpa lr = 1.5e-5 LB:0.775 
# [0.2616716774835217, 0.2830840063208972,  0.22212529014537136] train data only  lr = 1.5e-5

# lr = 1.5e-5
# [0.2720036358349279, 0.32137186476052365, 0.22517818749821006, 0.27430064060954523, 0.2065113527196303] LB:764

# all acc_step 4 lr 1e-05 wd 0.001 400-135 seed 42 msd 8
#[0.2674649548349373, 0.30405822478353256, 0.22417820907238017, 0.27306132025548135, 0.19995228248329291] chaii3-1006
# hindi xsquid acc_step 2 lr 1e-05 wd 0.001 400-135 seed 42 msd 8
#[0.2717109649169748, 0.29633474438177754, 0.22632861237664936, 0.2791072048015778, 0.19286308680928477]  LB: 0.758 ?

